# Mini-exercice (obligatoire) — Ablation LSTM vs GRU (2 epochs)

Objectif : comparer **LSTM vs GRU** à architecture comparable, sur **2 epochs** uniquement (juste pour observer l'effet).

On garde :
- même vocabulaire (`MAX_WORDS`)
- même longueur (`MAX_LEN`)
- même embedding dim
- mêmes hyperparams d'optimisation

On compare rapidement : accuracy + ROC-AUC sur validation.


## 1) Imports & Setup

In [1]:
from pathlib import Path
import sys

import numpy as np
import tensorflow as tf

cwd = Path.cwd().resolve()
PROJECT_DIR = None
for p in [cwd] + list(cwd.parents):
    if (p / 'src').exists():
        PROJECT_DIR = p
        break
if PROJECT_DIR is None:
    raise RuntimeError(f"Could not find project root containing 'src' starting from: {cwd}")
sys.path.insert(0, str(PROJECT_DIR))

from src.text_preprocessing import TextPreprocessor
from src.model_architecture import ModelConfig, build_bilstm_model, build_gru_model

print('Project dir:', PROJECT_DIR)
print('Python:', sys.version)
print('TensorFlow:', tf.__version__)

Project dir: C:\Users\bello\Documents\data-science-portfolio\02_DL_NLP_Sentiment
Python: 3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]
TensorFlow: 2.13.0


## 2) Charger IMDB (vectorisé + padding)

In [2]:
MAX_WORDS = 10_000
MAX_LEN = 200
VAL_SIZE = 5_000

pre = TextPreprocessor(max_words=MAX_WORDS, max_len=MAX_LEN)
data = pre.load_imdb_text(validation_size=VAL_SIZE, seed=42)

X_train, y_train = data.X_train, data.y_train
X_val, y_val = data.X_val, data.y_val

print('X_train:', X_train.shape, X_train.dtype)
print('X_val  :', X_val.shape, X_val.dtype)

X_train: (20000, 200) int64
X_val  : (5000, 200) int64


## 3) Config commune

In [3]:
cfg = ModelConfig(
    vocab_size=MAX_WORDS,
    max_len=MAX_LEN,
    embedding_dim=128,
    rnn_units=64,
    dropout=0.3,
)
cfg

ModelConfig(vocab_size=10000, max_len=200, embedding_dim=128, rnn_units=64, dropout=0.3)

## 4) Train 2 epochs — BiLSTM

In [4]:
bilstm = build_bilstm_model(cfg)
hist_bilstm = bilstm.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=2,
    batch_size=128,
    verbose=1,
)
bilstm_metrics = bilstm.evaluate(X_val, y_val, verbose=0)
print('BiLSTM val:', dict(zip(bilstm.metrics_names, [float(x) for x in bilstm_metrics])))

Epoch 1/2
157/157 [==============================] - 168s 1s/step - loss: 0.5132 - accuracy: 0.7412 - roc_auc: 0.8266 - val_loss: 0.3322 - val_accuracy: 0.8650 - val_roc_auc: 0.9352
Epoch 2/2
157/157 [==============================] - 156s 997ms/step - loss: 0.3008 - accuracy: 0.8832 - roc_auc: 0.9436 - val_loss: 0.3083 - val_accuracy: 0.8740 - val_roc_auc: 0.9419
BiLSTM val: {'loss': 0.30834636092185974, 'accuracy': 0.8740000128746033, 'roc_auc': 0.9418541193008423}


## 5) Train 2 epochs — BiGRU

In [5]:
bigru = build_gru_model(cfg)
hist_bigru = bigru.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=2,
    batch_size=128,
    verbose=1,
)
bigru_metrics = bigru.evaluate(X_val, y_val, verbose=0)
print('BiGRU val:', dict(zip(bigru.metrics_names, [float(x) for x in bigru_metrics])))

Epoch 1/2
157/157 [==============================] - 184s 1s/step - loss: 0.5604 - accuracy: 0.6813 - roc_auc: 0.7711 - val_loss: 0.4193 - val_accuracy: 0.8150 - val_roc_auc: 0.9018
Epoch 2/2
157/157 [==============================] - 248s 2s/step - loss: 0.3184 - accuracy: 0.8712 - roc_auc: 0.9374 - val_loss: 0.3230 - val_accuracy: 0.8684 - val_roc_auc: 0.9365
BiGRU val: {'loss': 0.3230196535587311, 'accuracy': 0.868399977684021, 'roc_auc': 0.9365116357803345}


## 6) Comparaison simple

In [6]:
def last_val(metric_name: str, hist) -> float:
    key = f'val_{metric_name}'
    return float(hist.history.get(key, [np.nan])[-1])

summary = {
    'BiLSTM': {
        'val_accuracy': last_val('accuracy', hist_bilstm),
        'val_roc_auc': last_val('roc_auc', hist_bilstm),
    },
    'BiGRU': {
        'val_accuracy': last_val('accuracy', hist_bigru),
        'val_roc_auc': last_val('roc_auc', hist_bigru),
    },
}
summary

{'BiLSTM': {'val_accuracy': 0.8740000128746033,
  'val_roc_auc': 0.9418541193008423},
 'BiGRU': {'val_accuracy': 0.868399977684021,
  'val_roc_auc': 0.9365116357803345}}

Donc sur 2 epochs, BiLSTM gagne légèrement (petit écart, normal que ce soit un peu bruité). On peut garder BiLSTM comme modèle “final” pour J3.

✅ Fin mini-exercice.

Note : sur 2 epochs, les résultats peuvent être bruyants. L'objectif est juste de voir une différence de dynamique (vitesse d'apprentissage/stabilité).